In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from backtester import StrategyRunner

# Load Backtest DFs

In [ ]:
squeeth_bt_df = pd.read_csv('strategy_dfs/squeeth_backtest_df.csv', index_col=0)
deribit_bt_df = pd.read_csv('strategy_dfs/deribit_backtest_df.csv', index_col=0)

# Backtest Analytics

In [ ]:
def backtest_analytics(output):
    output_df = pd.DataFrame(output).T
    
    pnl = output_df['unrealized_pnl']
    capital = output_df['capital']
    capital_for_rets = capital.replace(to_replace=0, method='ffill').shift(1)
    rets = pnl / capital_for_rets
    
    pnl.cumsum().plot(figsize=(10, 5))
    plt.ylabel('Cumulative $ PnL')
    plt.grid(True)
    plt.show()
    
    rets.cumsum().plot(figsize=(10, 5))
    plt.ylabel('Cumulative % Rets')
    plt.grid(True)
    plt.show()
    
    er = rets.mean()*365*24
    vol = rets.std()*np.sqrt(365*24)
    sharpe = er / vol
    print('ER: {:.2f}'.format(er))
    print('Vol: {:.2f}'.format(vol))
    print('Sharpe: {:.2f}'.format(sharpe))
        
    output_df['delta'].fillna(0).plot(figsize=(5, 3))
    plt.ylabel('Portfolio Delta')
    plt.grid(True)
    plt.show()
    
    output_df['gamma'].fillna(0).plot(figsize=(5, 3))
    plt.ylabel('Portfolio Gamma')
    plt.grid(True)
    plt.show()

    output_df['vega'].fillna(0).plot(figsize=(5, 3))
    plt.ylabel('Portfolio Vega (per 1% move)')
    plt.grid(True)
    plt.show()
    
    (output_df['theta'].fillna(0) / 24.).plot(figsize=(5, 3))
    plt.ylabel('Portfolio Theta (hourly)')
    plt.grid(True)
    plt.show()

    return {
        'rets': rets, 
        'pnl': pnl, 
        'stats': {'er': er, 'vol': vol, 'sharpe': sharpe}
    }

In [ ]:
def plot_diff_settings(output, setting_name='IV Ratio Trigger'):
    columns = list(output.keys())
    pnl_df = pd.DataFrame({key: val['analytics']['pnl'] for key, val in output.items()}, columns=columns)
    rets_df = pd.DataFrame({key: val['analytics']['rets'] for key, val in output.items()}, columns=columns)
    
    pnl_df.cumsum().plot(figsize=(10, 5))
    plt.ylabel('Cumulative $ PnL')
    plt.legend(title=setting_name)
    plt.grid(True)
    plt.show()
    
    rets_df.cumsum().plot(figsize=(10, 5))
    plt.ylabel('Cumulative % Rets')
    plt.legend(title=setting_name)
    plt.grid(True)
    plt.show()

    stats_df = pd.DataFrame({key: val['analytics']['stats'] for key, val in output.items()}, columns=columns)
    return pnl_df, rets_df, stats_df
    

In [ ]:
def convert_by_asset_to_df(output, item = 'unrealized_pnl_by_asset'):
    output_dict = {time: output[time][item] for time in output.keys()}
    return pd.DataFrame(output_dict).T

# Different IV Ratio Triggers

In [ ]:
def run_different_iv_triggers(
    squeeth_bt_df,
    deribit_bt_df,
    vals=[0, 1, 1.1, 1.2, 1.3, 1.4]
):
    return_dict = {}
    for val in vals:
        print('Running iv trigger = {}'.format(val))
        sr = StrategyRunner(squeeth_bt_df, deribit_bt_df, ivol_ratio_trigger=val)
        output = sr.run()
        return_dict[val] = {'output': output, 'analytics': backtest_analytics(output)}
    return return_dict

In [ ]:
iv_ratio_results = run_different_iv_triggers(squeeth_bt_df, deribit_bt_df)

In [ ]:
iv_ratio_results_formatted = plot_diff_settings(iv_ratio_results, 'IV Ratio Trigger')

# Different EWMA Smooth

In [ ]:
def run_different_ewma_halflifes(
    squeeth_bt_df,
    deribit_bt_df,
    vals=[0.0001, 1, 3, 5, 10, 20, 50]
):
    return_dict = {}
    for val in vals:
        print('Running EWMA halflife = {} hours'.format(val))
        sr = StrategyRunner(squeeth_bt_df, deribit_bt_df, ivol_smooth_halflife=val)
        output = sr.run()
        return_dict[val] = {'output': output, 'analytics': backtest_analytics(output)}
    return return_dict

In [ ]:
ewma_results = run_different_ewma_halflifes(squeeth_bt_df, deribit_bt_df)

In [ ]:
ewma_results_formatted = plot_diff_settings(ewma_results, 'EMWA Halflife')

# Run Different Tcost Multipliers

In [ ]:
def run_different_tcost_multipliers(
    squeeth_bt_df,
    deribit_bt_df,
    vals=[0.5, 1, 1.5, 2, 3]
):
    return_dict = {}
    for val in vals:
        print('Running tcost multiplier = {}'.format(val))
        sr = StrategyRunner(squeeth_bt_df, deribit_bt_df, tcost_multiplier=val)
        output = sr.run()
        return_dict[val] = {'output': output, 'analytics': backtest_analytics(output)}
    return return_dict

In [ ]:
tcost_results = run_different_tcost_multipliers(squeeth_bt_df, deribit_bt_df)

In [ ]:
tcost_results_formatted = plot_diff_settings(tcost_results, 'Tcost Multiplier')